In [ ]:
import pandas as pd
import numpy as np
import reportsutils as utils
from matplotlib import pyplot



In [ ]:
countries_list = ["Germany", "Spain", "Austria", "Switzerland", "Italy", "France"]

In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

column_name = 'Confirmed_Change'

for country in countries_list:
    country_df = utils.get_country_report(country)
    #country_df = country_df[country_df.index >pd.to_datetime('01-05-2020',dayfirst=True)]
    
    country_df[column_name+'_SMA7'] = country_df[column_name].rolling(window=7).mean()
    ax.bar(country_df.index, country_df[column_name].values, label=country, alpha=0.1)
    ax.plot(country_df.index, country_df[column_name+'_SMA7'].values, label=country + '-SMA7') 

ax.legend()
#ax.set_ylim(top=800)
pyplot.yscale("log")
pyplot.show()

In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

for country in utils.get_countries():
    country_df = utils.get_country_report(country)
    #country_df = country_df[country_df.Date >pd.to_datetime('01-05-2020',dayfirst=True)]

    country_df['Confirmed_Change_Normalized'] = utils.normalize(country_df['Confirmed_Change'])
    #condition_row = country_df[country_df.Confirmed_Change_Normalized >= 0.05].iloc[0]
    #country_df = country_df[condition_row.name:]
    
    country_df['Confirmed_Change_SMA7'] = utils.normalize(country_df['Confirmed_Change'].rolling(window=7).mean())
    country_df['Confirmed_Change_SMA14'] = utils.normalize(country_df['Confirmed_Change'].rolling(window=14).mean())

    #country_df.dropna(inplace=True, subset=['Confirmed_Change_SMA14'])

    #country_df.reset_index(inplace=True, drop=True)

    #ax.bar(country_df.index, country_df['Confirmed_Change_Normalized'].values, label=country, alpha=0.1)
    ax.plot(country_df.index, country_df['Confirmed_Change_SMA14'].values, alpha = 0.05, color = "Blue")
    #if(country in countries_list):
    #    ax.plot(country_df.index, country_df['Confirmed_Change_SMA14'].values, color = "Green")

df = utils.get_countries_report_by_column("Confirmed_Change")
df.fillna(0, inplace=True)
df['Total'] = df.sum(axis =1)
df['Total_Normalized'] = utils.normalize(df.Total)
df['Total_SMA'] = utils.normalize(df.Total.rolling(window=14).mean())

ax.plot(df.index, df.Total_SMA.values, color = "Red", label = "Среднее по миру", alpha = 0.7)

ax.set_ylim(bottom=0, top = 1)
pyplot.title("Нормализованное количество заболевших в день")
ax.legend(loc='upper left')
pyplot.show()


In [ ]:
column_name = "Confirmed_Change"
start_date = pd.to_datetime('01-04-2020', dayfirst=True)

moscow_df = utils.get_region_report("Russia", "Москва")[start_date:]
regions_df = utils.get_regions_report_by_column("Russia", column_name, exclude=["Москва"], start_date=start_date)


In [ ]:
pyplot.figure(figsize=(18,10))

ax = pyplot.subplot(111)
ax.xaxis_date()

regions_df['Total'] = 0
regions_df['Total'] = regions_df.sum(axis = 1)

ax.bar(regions_df.index, regions_df['Total'].values, label = 'Регионы', alpha=0.3)
ax.plot(regions_df.index, regions_df['Total'].rolling(window=7).mean(), label= 'Регионы-SMA7')

ax.bar(moscow_df.index, moscow_df[column_name].values, label = 'Москва', alpha = 0.3)
ax.plot(moscow_df.index, moscow_df[column_name].rolling(window=7).mean(), label= 'Москва-SMA7')


utils.draw_key_russian_dates_on_plot(ax)
#ax.axhspan(5700, 6100, 0.75, 0.9, alpha = 0.4, color = 'Grey')
#ax.axhspan(500, 700, 0.78, 0.95, alpha = 0.4, color = 'Grey', label = "Неизменное число заболевших")

#ax.set_ylim(bottom=5000, top = 6500)
ax.legend(loc='upper left')
ax.set_title('Количество заболевших в день в Москве и остальной России')
ax.grid(axis='y', color='black', linestyle='dashed', alpha=0.4)
pyplot.show()

In [ ]:
pyplot.figure(figsize=(20,15))

ax = pyplot.subplot(111)
ax.xaxis_date()

column_name = 'Recovered_Change'

for region_name in ["Санкт-Петербург", "Ростовская обл.", "Краснодарский край", "Хабаровский край"]:
    region_df = utils.get_region_report("Russia", region_name)[start_date:]

    ax.bar(region_df.index, region_df[column_name].values, label = region_name+'-'+column_name, alpha=0.3)
    ax.plot(
        region_df.index,
        region_df[column_name].rolling(window=7).mean().values,
        label = region_name+'-'+column_name+'-SMA7',
        alpha=0.9)

utils.draw_key_russian_dates_on_plot(ax)

#ax.set_ylim(bottom=5000, top = 6500)
ax.legend(loc='upper left')
pyplot.show()

In [ ]:
regions = list(utils.get_country_regions("Russia"))

i=1

for region in regions:
    pyplot.figure(figsize=(18*2,10))
    ax = pyplot.subplot(1,2,1)
    dfdf = utils.get_region_report("Russia", region)[60:]

    index = dfdf.index

    confirmed = dfdf.Confirmed_Change
    confirmed_SMA7 = confirmed.rolling(window=7).mean()

    recovered = dfdf.Recovered_Change
    recovered_SMA7 = recovered.rolling(window=7).mean()

    deaths = dfdf.Deaths_Change
    deaths_SMA7 = deaths.rolling(window=7).mean()

    active = dfdf.Active
    active_SMA7 = active.rolling(window=7).mean()

    ax.xaxis_date()

    ax.bar(index, confirmed, label = 'Заболевшие', alpha=0.3)
    ax.plot(index, confirmed_SMA7, label= 'Заболевшие-SMA7')

    ax.bar(index, recovered, label = 'Выздоровевшие', alpha = 0.3)
    ax.plot(index, recovered_SMA7, label= 'Выздоровевшие-SMA7')

    ax.bar(index, deaths, label = 'Смерти', alpha = 0.3, bottom = recovered)
    ax.plot(index, recovered_SMA7+deaths_SMA7, label= 'Смерти-SMA7')

    utils.draw_key_russian_dates_on_plot(ax)

    ax.set_ylim(bottom=0)
    ax.legend(loc='upper left')
    ax.grid(axis='y', color='black', linestyle='dashed', alpha=0.4)

    ax = pyplot.subplot(1,2,2)

    ax.xaxis_date()

    ax.bar(index, active, label = 'Больные', alpha=0.3)
    ax.plot(index, active_SMA7, label= 'Больные-SMA7')

    utils.draw_key_russian_dates_on_plot(ax)

    ax.set_ylim(bottom=0)
    ax.legend(loc='upper left')
    ax.grid(axis='y', color='black', linestyle='dashed', alpha=0.4)


    pyplot.suptitle(region)
    pyplot.show()

    i+=1



In [ ]:
dt = pd.DataFrame()


sub1 = pd.DataFrame([2,3,8], [1,2,3], columns = ["Index"])
sub2 = pd.DataFrame([0,1,3,6], [3,4,5,6], columns= ["Idx"])

In [ ]:
sub1

In [ ]:
sub2

In [ ]:
dt["sub1"] = sub1.Index
pd.concat([sub1.Index,sub2.Idx], axis=1)

In [ ]:
sub2